In [7]:
import scipy.io as sio
from scipy.misc import imread
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from .. import datasets
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_curve
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split

ValueError: attempted relative import beyond top-level package

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\GPDSSyntheticSignatures4k\\bmgnet_features'

In [3]:
user_kernel='rbf'

In [4]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

# MODEL SELECTION & TRAINING

In [5]:
dev_exp_ratio = 0.30
sorted_id_list = np.sort(data_f['user_id'].unique())
dev_exp_splitter=int(len(sorted_id_list)*dev_exp_ratio)
dev_val_user_ids = sorted_id_list[:dev_exp_splitter]
exp_user_ids = sorted_id_list[dev_exp_splitter:]

In [6]:
dev_val_user_ids.shape

(1200,)

In [7]:
exp_user_ids.shape

(2800,)

In [10]:
fakes_preds = []
gens_preds = []

for fold in np.arange(0,10):
    # assert len(dev_val_user_ids)==581
    np.random.shuffle(dev_val_user_ids)
    dev_user_ids = dev_val_user_ids[0:1000]
    validation_user_ids = dev_val_user_ids[1000:len(dev_val_user_ids)]
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5, test_size=0.5)

    dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
    dev_vf = visual_f.loc[dev_df.index]
    val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
    val_vf = visual_f.loc[val_df.index]

    dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
    dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
    dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(train_idx)]
    dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(test_idx)]
    
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
    val_df_gen = val_df.loc[val_df['fakeness']==0]
    val_df_fake = val_df.loc[val_df['fakeness']==1]
    val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(train_idx)]
    val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(test_idx)]

    for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
        clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel=user_kernel)
        y_train = (pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen.loc[dev_df_gen['user_id']!=user_id]]))['user_id']==user_id
        X_train = visual_f.loc[y_train.index]  
        clf.fit(X_train, y_train)
        y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
        X_valid_f = visual_f.loc[y_valid_fakes.index]
        fakes_preds.append(clf.decision_function(X_valid_f))
        y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
        X_valid_g = visual_f.loc[y_valid_gens.index]
        gens_preds.append(clf.decision_function(X_valid_g))

c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

  0%|4                                                                                 | 1/200 [00:05<18:20,  5.53s/it]

  1%|8                                                                                 | 2/200 [00:09<16:15,  4.93s/it]

  2%|#2                                                                                | 3/200 [00:15<17:57,  5.47s/it]

  2%|#6                                                                                | 4/200 [00:21<17:52,  5.47s/it]

  2%|##                                                                                | 5/200 [00:27<18:44,  5.77s/it]

  3%|##4                                                

 66%|####################################################8                           | 132/200 [12:12<07:48,  6.88s/it]

 66%|#####################################################2                          | 133/200 [12:16<06:44,  6.04s/it]

 67%|#####################################################6                          | 134/200 [12:21<06:17,  5.71s/it]

 68%|######################################################                          | 135/200 [12:26<05:47,  5.35s/it]

 68%|######################################################4                         | 136/200 [12:32<05:56,  5.58s/it]

 68%|######################################################8                         | 137/200 [12:36<05:32,  5.28s/it]

 69%|#######################################################2                        | 138/200 [12:41<05:13,  5.05s/it]

 70%|#######################################################6                        | 139/200 [12:48<05:38,  5.54s/it]

 70%|###########################

 32%|#########################5                                                       | 63/200 [05:29<11:33,  5.07s/it]

 32%|#########################9                                                       | 64/200 [05:34<11:15,  4.96s/it]

 32%|##########################3                                                      | 65/200 [05:40<11:31,  5.12s/it]

 33%|##########################7                                                      | 66/200 [05:46<12:16,  5.50s/it]

 34%|###########################1                                                     | 67/200 [05:50<11:02,  4.98s/it]

 34%|###########################5                                                     | 68/200 [05:55<11:02,  5.02s/it]

 34%|###########################9                                                     | 69/200 [06:01<11:36,  5.31s/it]

 35%|############################3                                                    | 70/200 [06:06<11:07,  5.14s/it]

 36%|###########################

 98%|##############################################################################8 | 197/200 [17:13<00:15,  5.05s/it]

 99%|###############################################################################2| 198/200 [17:19<00:10,  5.26s/it]

100%|###############################################################################6| 199/200 [17:24<00:05,  5.30s/it]

100%|################################################################################| 200/200 [17:29<00:00,  5.16s/it]c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

  0%|4                                                                                 | 1/200 [00:05<19:48,  5.97s/it]

  1%|8                                                    

 64%|###################################################2                            | 128/200 [11:04<06:15,  5.21s/it]

 64%|###################################################6                            | 129/200 [11:11<06:36,  5.59s/it]

 65%|####################################################                            | 130/200 [11:16<06:32,  5.60s/it]

 66%|####################################################4                           | 131/200 [11:21<06:14,  5.43s/it]

 66%|####################################################8                           | 132/200 [11:26<05:50,  5.15s/it]

 66%|#####################################################2                          | 133/200 [11:31<05:42,  5.12s/it]

 67%|#####################################################6                          | 134/200 [11:35<05:09,  4.69s/it]

 68%|######################################################                          | 135/200 [11:42<05:51,  5.40s/it]

 68%|###########################

...

 26%|####################6                                                            | 51/200 [04:37<13:33,  5.46s/it]

 26%|#####################                                                            | 52/200 [04:42<13:48,  5.60s/it]

 26%|#####################4                                                           | 53/200 [04:46<12:06,  4.94s/it]

 27%|#####################8                                                           | 54/200 [04:52<12:59,  5.34s/it]

 28%|######################2                                                          | 55/200 [04:58<13:32,  5.61s/it]

 28%|######################6                                                          | 56/200 [05:05<14:34,  6.07s/it]

 28%|#######################                                                          | 57/200 [05:10<13:15,  5.57s/it]

 29%|#######################4                                                         | 58/200 [05:16<13:30,  5.71s/it]

 30%|#######################8   

 92%|##########################################################################      | 185/200 [16:31<01:09,  4.62s/it]

 93%|##########################################################################4     | 186/200 [16:35<01:03,  4.52s/it]

 94%|##########################################################################8     | 187/200 [16:40<01:00,  4.63s/it]

 94%|###########################################################################1    | 188/200 [16:45<00:58,  4.87s/it]

 94%|###########################################################################6    | 189/200 [16:51<00:56,  5.13s/it]

 95%|############################################################################    | 190/200 [16:56<00:50,  5.09s/it]

 96%|############################################################################3   | 191/200 [17:01<00:44,  4.99s/it]

 96%|############################################################################8   | 192/200 [17:07<00:41,  5.17s/it]

 96%|###########################

 58%|##############################################4                                 | 116/200 [10:00<06:59,  5.00s/it]

 58%|##############################################8                                 | 117/200 [10:06<07:14,  5.24s/it]

 59%|###############################################1                                | 118/200 [10:11<07:15,  5.31s/it]

 60%|###############################################5                                | 119/200 [10:16<07:00,  5.19s/it]

 60%|################################################                                | 120/200 [10:21<06:47,  5.10s/it]

 60%|################################################4                               | 121/200 [10:26<06:47,  5.15s/it]

 61%|################################################8                               | 122/200 [10:31<06:20,  4.87s/it]

 62%|#################################################2                              | 123/200 [10:35<06:14,  4.86s/it]

 62%|###########################

 24%|###################                                                              | 47/200 [04:02<12:05,  4.74s/it]

 24%|###################4                                                             | 48/200 [04:09<13:45,  5.43s/it]

 24%|###################8                                                             | 49/200 [04:15<14:16,  5.67s/it]

 25%|####################2                                                            | 50/200 [04:20<13:18,  5.32s/it]

 26%|####################6                                                            | 51/200 [04:25<13:38,  5.49s/it]

 26%|#####################                                                            | 52/200 [04:30<13:03,  5.29s/it]

 26%|#####################4                                                           | 53/200 [04:36<12:58,  5.30s/it]

 27%|#####################8                                                           | 54/200 [04:40<11:59,  4.93s/it]

 28%|######################2    

 90%|########################################################################4       | 181/200 [15:39<01:40,  5.28s/it]

 91%|########################################################################8       | 182/200 [15:44<01:33,  5.17s/it]

 92%|#########################################################################2      | 183/200 [15:49<01:30,  5.33s/it]

 92%|#########################################################################6      | 184/200 [15:53<01:19,  4.98s/it]

 92%|##########################################################################      | 185/200 [15:59<01:16,  5.13s/it]

 93%|##########################################################################4     | 186/200 [16:04<01:11,  5.09s/it]

 94%|##########################################################################8     | 187/200 [16:09<01:07,  5.15s/it]

 94%|###########################################################################1    | 188/200 [16:14<01:02,  5.20s/it]

 94%|###########################

# GLOBAL THRESHOLD SELECTION

In [11]:
flat_fakes_preds = np.expand_dims(np.array([item for sublist in fakes_preds for item in sublist]),axis=1)
flat_gens_preds = np.expand_dims(np.array([item for sublist in gens_preds for item in sublist]),axis=1)
all_preds = np.vstack((flat_fakes_preds,flat_gens_preds))
all_labels = np.vstack((np.zeros((flat_fakes_preds.shape[0],1)),np.ones((flat_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_labels,all_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob : ', EER*100,'\nEER_Threshold_glob : ', eer_th)
glob_th = eer_th

EER_glob :  7.076666666666667 
EER_Threshold_glob :  0.047372913864139


In [12]:
assert len(fakes_preds)==len(gens_preds)
EER_accum=0
for idx,val in enumerate(fakes_preds):
    user_fakes_preds = np.expand_dims(np.array(fakes_preds[idx]),axis=1)
    user_gens_preds = np.expand_dims(np.array(gens_preds[idx]),axis=1)
    all_user_preds = np.vstack((user_fakes_preds,user_gens_preds))
    all_user_labels = np.vstack((np.zeros((user_fakes_preds.shape[0],1)),np.ones((user_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_labels,all_user_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user : ', (EER_accum*100)/len(fakes_preds)) 

EER_user :  3.371666666666672


In [13]:
print(glob_th)

0.047372913864139


# TRAIN AND TEST ON THE EXPLOITATION SET

In [14]:
test_gens_preds = []
test_fakes_preds = []

train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
exp_df = data_f.loc[data_f['user_id'].isin(exp_user_ids)]
exp_vf = visual_f.loc[exp_df.index]
exp_df_gen = exp_df.loc[exp_df['fakeness']==0]
exp_df_fake = exp_df.loc[exp_df['fakeness']==1]
exp_df_fake_10 = exp_df_fake.loc[exp_df_fake['sig_id'].isin(choice(np.arange(1,31),10,replace=False))]
exp_df_gen_12 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(train_idx)]
exp_df_valid_gen_10 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(test_idx[0:10])]
exp_df_gen_10 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(train_idx[:-2])]

dev_val_df = data_f.loc[data_f['user_id'].isin(dev_val_user_ids)]
dev_val_vf = visual_f.loc[dev_val_df.index]
dev_val_df_gen = dev_val_df.loc[dev_val_df['fakeness']==0]
dev_val_df_valid_gen_14 = dev_val_df_gen.loc[dev_val_df_gen['sig_id'].isin(choice(np.arange(1,25),14,replace=False))]

c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
exp_df_gen_10

,path,user_id,sig_id,fakeness
17334,c-1201-01.jpg,1201,1,0
17336,c-1201-03.jpg,1201,3,0
17338,c-1201-05.jpg,1201,5,0
17341,c-1201-08.jpg,1201,8,0
17342,c-1201-09.jpg,1201,9,0
17343,c-1201-10.jpg,1201,10,0
17350,c-1201-17.jpg,1201,17,0
17351,c-1201-18.jpg,1201,18,0
17356,c-1201-23.jpg,1201,23,0
17357,c-1201-24.jpg,1201,24,0


In [16]:
for user_id in tqdm.tqdm(exp_user_ids, ascii=True):
    clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel=user_kernel)
    y_train = (pd.concat([exp_df_gen_10.loc[exp_df_gen_10['user_id']==user_id],dev_val_df_valid_gen_14.loc[dev_val_df_valid_gen_14['user_id']!=user_id]]))['user_id']==user_id
    X_train = visual_f.loc[y_train.index]  
    clf.fit(X_train, y_train)
    y_valid_fakes = exp_df_fake_10.loc[(exp_df_fake_10['user_id']==user_id)]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    test_fakes_preds.append(clf.decision_function(X_valid_f))
    y_valid_gens = exp_df_valid_gen_10.loc[exp_df_valid_gen_10['user_id']==user_id]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    test_gens_preds.append(clf.decision_function(X_valid_g))



  0%|                                                                                         | 0/2800 [00:00<?, ?it/s]

  0%|                                                                               | 1/2800 [00:03<2:28:41,  3.19s/it]

  0%|                                                                               | 2/2800 [00:06<2:24:35,  3.10s/it]

  0%|                                                                               | 3/2800 [00:09<2:31:46,  3.26s/it]

  0%|1                                                                              | 4/2800 [00:13<2:38:26,  3.40s/it]

  0%|1                                                                              | 5/2800 [00:15<2:23:56,  3.09s/it]

  0%|1                                                                              | 6/2800 [00:18<2:24:55,  3.11s/it]

  0%|1                                                                              | 7/2800 [00:23<2:39:01,  3.42s/it]

  0%|2                        

  5%|###6                                                                         | 134/2800 [07:08<2:25:56,  3.28s/it]

  5%|###7                                                                         | 135/2800 [07:12<2:30:49,  3.40s/it]

  5%|###7                                                                         | 136/2800 [07:15<2:31:28,  3.41s/it]

  5%|###7                                                                         | 137/2800 [07:18<2:20:51,  3.17s/it]

  5%|###7                                                                         | 138/2800 [07:21<2:29:39,  3.37s/it]

  5%|###8                                                                         | 139/2800 [07:25<2:27:05,  3.32s/it]

  5%|###8                                                                         | 140/2800 [07:28<2:21:41,  3.20s/it]

  5%|###8                                                                         | 141/2800 [07:32<2:31:31,  3.42s/it]

  5%|###9                       

 10%|#######3                                                                     | 268/2800 [14:20<2:27:11,  3.49s/it]

 10%|#######3                                                                     | 269/2800 [14:23<2:19:12,  3.30s/it]

 10%|#######4                                                                     | 270/2800 [14:26<2:14:55,  3.20s/it]

 10%|#######4                                                                     | 271/2800 [14:29<2:12:59,  3.16s/it]

 10%|#######4                                                                     | 272/2800 [14:33<2:23:42,  3.41s/it]

 10%|#######5                                                                     | 273/2800 [14:36<2:18:50,  3.30s/it]

 10%|#######5                                                                     | 274/2800 [14:40<2:23:39,  3.41s/it]

 10%|#######5                                                                     | 275/2800 [14:43<2:20:36,  3.34s/it]

 10%|#######5                   

 14%|###########                                                                  | 402/2800 [21:24<2:01:21,  3.04s/it]

 14%|###########                                                                  | 403/2800 [21:28<2:12:03,  3.31s/it]

 14%|###########1                                                                 | 404/2800 [21:31<2:09:43,  3.25s/it]

 14%|###########1                                                                 | 405/2800 [21:35<2:09:49,  3.25s/it]

 14%|###########1                                                                 | 406/2800 [21:38<2:11:22,  3.29s/it]

 15%|###########1                                                                 | 407/2800 [21:41<2:08:09,  3.21s/it]

 15%|###########2                                                                 | 408/2800 [21:43<1:56:21,  2.92s/it]

 15%|###########2                                                                 | 409/2800 [21:46<1:57:14,  2.94s/it]

 15%|###########2               

 19%|##############7                                                              | 536/2800 [28:30<1:59:20,  3.16s/it]

 19%|##############7                                                              | 537/2800 [28:34<1:59:26,  3.17s/it]

 19%|##############7                                                              | 538/2800 [28:36<1:49:06,  2.89s/it]

 19%|##############8                                                              | 539/2800 [28:39<1:51:49,  2.97s/it]

 19%|##############8                                                              | 540/2800 [28:42<1:55:22,  3.06s/it]

 19%|##############8                                                              | 541/2800 [28:46<2:06:23,  3.36s/it]

 19%|##############9                                                              | 542/2800 [28:50<2:06:34,  3.36s/it]

 19%|##############9                                                              | 543/2800 [28:53<2:04:59,  3.32s/it]

 19%|##############9            

...

 77%|##########################################################1                 | 2144/2800 [1:59:22<41:08,  3.76s/it]

 77%|##########################################################2                 | 2145/2800 [1:59:25<39:16,  3.60s/it]

 77%|##########################################################2                 | 2146/2800 [1:59:28<38:22,  3.52s/it]

 77%|##########################################################2                 | 2147/2800 [1:59:32<39:07,  3.59s/it]

 77%|##########################################################3                 | 2148/2800 [1:59:36<38:49,  3.57s/it]

 77%|##########################################################3                 | 2149/2800 [1:59:40<39:53,  3.68s/it]

 77%|##########################################################3                 | 2150/2800 [1:59:44<41:27,  3.83s/it]

 77%|##########################################################3                 | 2151/2800 [1:59:47<39:27,  3.65s/it]

 77%|###########################

 81%|#############################################################8              | 2278/2800 [2:07:49<35:33,  4.09s/it]

 81%|#############################################################8              | 2279/2800 [2:07:52<33:09,  3.82s/it]

 81%|#############################################################8              | 2280/2800 [2:07:57<35:17,  4.07s/it]

 81%|#############################################################9              | 2281/2800 [2:08:00<34:30,  3.99s/it]

 82%|#############################################################9              | 2282/2800 [2:08:04<32:34,  3.77s/it]

 82%|#############################################################9              | 2283/2800 [2:08:08<33:41,  3.91s/it]

 82%|#############################################################9              | 2284/2800 [2:08:11<31:32,  3.67s/it]

 82%|##############################################################              | 2285/2800 [2:08:14<30:28,  3.55s/it]

 82%|###########################

 86%|#################################################################4          | 2412/2800 [2:16:02<23:45,  3.67s/it]

 86%|#################################################################4          | 2413/2800 [2:16:05<22:11,  3.44s/it]

 86%|#################################################################5          | 2414/2800 [2:16:09<22:51,  3.55s/it]

 86%|#################################################################5          | 2415/2800 [2:16:13<22:51,  3.56s/it]

 86%|#################################################################5          | 2416/2800 [2:16:16<22:47,  3.56s/it]

 86%|#################################################################6          | 2417/2800 [2:16:21<25:16,  3.96s/it]

 86%|#################################################################6          | 2418/2800 [2:16:24<23:41,  3.72s/it]

 86%|#################################################################6          | 2419/2800 [2:16:28<23:40,  3.73s/it]

 86%|###########################

 91%|#####################################################################1      | 2546/2800 [2:24:23<15:56,  3.77s/it]

 91%|#####################################################################1      | 2547/2800 [2:24:26<15:39,  3.71s/it]

 91%|#####################################################################1      | 2548/2800 [2:24:30<15:30,  3.69s/it]

 91%|#####################################################################1      | 2549/2800 [2:24:34<15:26,  3.69s/it]

 91%|#####################################################################2      | 2550/2800 [2:24:38<15:41,  3.77s/it]

 91%|#####################################################################2      | 2551/2800 [2:24:41<15:15,  3.68s/it]

 91%|#####################################################################2      | 2552/2800 [2:24:45<15:42,  3.80s/it]

 91%|#####################################################################2      | 2553/2800 [2:24:49<15:03,  3.66s/it]

 91%|###########################

 96%|########################################################################7   | 2680/2800 [2:32:43<07:21,  3.68s/it]

 96%|########################################################################7   | 2681/2800 [2:32:46<06:50,  3.45s/it]

 96%|########################################################################7   | 2682/2800 [2:32:48<06:21,  3.23s/it]

 96%|########################################################################8   | 2683/2800 [2:32:53<07:00,  3.59s/it]

 96%|########################################################################8   | 2684/2800 [2:32:55<06:21,  3.29s/it]

 96%|########################################################################8   | 2685/2800 [2:32:58<05:50,  3.05s/it]

 96%|########################################################################9   | 2686/2800 [2:33:00<05:35,  2.95s/it]

 96%|########################################################################9   | 2687/2800 [2:33:03<05:21,  2.85s/it]

 96%|###########################

In [17]:
flat_test_fakes_preds = np.expand_dims(np.array([item for sublist in test_fakes_preds for item in sublist]),axis=1)
flat_test_gens_preds = np.expand_dims(np.array([item for sublist in test_gens_preds for item in sublist]),axis=1)
print("____At the EER threshold decided on the Validation set____")
print("FRR : ",(1-len(flat_test_gens_preds[flat_test_gens_preds>=glob_th])/len(flat_test_gens_preds))*100)
print("FARskilled : ",(1-len(flat_test_fakes_preds[flat_test_fakes_preds<glob_th])/len(flat_test_fakes_preds))*100)

____At the EER threshold decided on the Validation set____
FRR :  7.171428571428573
FARskilled :  6.832142857142854


In [18]:
all_test_preds = np.vstack((flat_test_fakes_preds,flat_test_gens_preds))
all_test_labels = np.vstack((np.zeros((flat_test_fakes_preds.shape[0],1)),np.ones((flat_test_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_test_labels,all_test_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob for test set: ', EER*100,'\nEER_Threshold_glob for test set: ', eer_th)

EER_glob for test set:  6.957142857142857 
EER_Threshold_glob for test set:  0.040182862782056405


In [19]:
assert len(test_fakes_preds)==len(test_gens_preds)
EER_accum=0
for idx,val in enumerate(test_fakes_preds):
    user_test_fakes_preds = np.expand_dims(np.array(test_fakes_preds[idx]),axis=1)
    user_test_gens_preds = np.expand_dims(np.array(test_gens_preds[idx]),axis=1)
    all_user_test_preds = np.vstack((user_test_fakes_preds,user_test_gens_preds))
    all_user_test_labels = np.vstack((np.zeros((user_test_fakes_preds.shape[0],1)),np.ones((user_test_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_test_labels,all_user_test_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user for test set : ', (EER_accum*100)/len(test_fakes_preds))

EER_user for test set :  2.517857142857152
